# Init & Login

In [1]:
from math import ceil
from random import randrange
from time import sleep

import spotipy
import spotipy.util as util
from IPython.display import clear_output

## Client Info ##
CLIENT_ID     = ""
CLIENT_SECRET = ""
CLIENT_SCOPE  = "user-follow-modify playlist-modify-private playlist-modify-public"
USER_NAME     = "31ytgsr7wdmiaroy77msqpiupdsi"
REDIR_URI     = "https://github.com/jwatson-CO-edu/yt_shuffle_so_good"
AUTH_URL      = 'https://accounts.spotify.com/api/token'
BASE_URL      = 'https://api.spotify.com/v1/'
## API Info ##
_RESPONSE_LIMIT = 100

with open( "../keys/spot_ID.txt" , 'r' ) as f:
    CLIENT_ID = f.readlines()[0].strip()

with open( "../keys/spot_SECRET.txt" , 'r' ) as f:
    CLIENT_SECRET = f.readlines()[0].strip()

token = None
token = util.prompt_for_user_token(
    username      = USER_NAME,
    scope         = CLIENT_SCOPE,
    client_id     = CLIENT_ID,
    client_secret = CLIENT_SECRET,
    redirect_uri  = REDIR_URI
)

print( token )

spot = spotipy.Spotify( auth = token )
clear_output( wait = True )
sleep( 2 )
print( "TOKEN OBTAINED" )

TOKEN OBTAINED


# Playlist Functions

In [2]:

def get_playlist_length( playlist_ID ):
    """ Get the number of total tracks in the playlist """
    response = spot.user_playlist_tracks(
        CLIENT_ID, 
        playlist_ID, 
        fields = 'items,uri,name,id,total', 
        limit  = _RESPONSE_LIMIT
    )
    return response['total']


def reorder_playlist_by_chunks( playlist_ID, chunkSize, Nmoves = None, verbose = False ):
    """ Shuffle the playlist a chunk at a time """
    Ntracks = get_playlist_length( playlist_ID )
    Nchunks = int( ceil( Ntracks / chunkSize ) )
    if Nmoves is None:
        Nmoves = int( Ntracks / chunkSize )
    for i in range( Nmoves ):
        bgnDex   = randrange(0,Nchunks) * chunkSize
        moveSize = min( chunkSize, Ntracks - bgnDex - 1 )
        sendDex  = randrange(0,Ntracks-1)
        try:
            response = spot.playlist_reorder_items(
                playlist_ID, 
                bgnDex, 
                sendDex, 
                range_length = chunkSize
            )
            if verbose:
                print( f"Iteration {i+1}, {bgnDex}-{bgnDex+moveSize} --> {sendDex}:\n\t{response}" )
        except spotipy.SpotifyException as ex:
            if verbose:
                print( f"ERROR, {bgnDex}-{bgnDex+moveSize} --> {sendDex}:\n\t{ex}" )


def complete_reorder_entire_playlist( playlist_ID, verbose = False, phasePause_s = 5 ):
    """ Completely shuffle the playlist in decreasing chunks """
    Ntracks = get_playlist_length( playlist_ID )
    ## Phase 1: Chunks of 100 ##
    if verbose:
        print("\nPhase 1: Chunks of 100\n")
    reorder_playlist_by_chunks( playlist_ID, 100, verbose = verbose )
    sleep( phasePause_s )
    ## Phase 2: Chunks of 10 ##
    if verbose:
        print("\nPhase 2: Chunks of 10\n")
    reorder_playlist_by_chunks( playlist_ID, 10, verbose = verbose )
    sleep( phasePause_s )
    ## Phase 3: Chunks of 5, 1/2 ##
    if verbose:
        print("\nPhase 3: Chunks of 5, 1/2\n")
    Nmoves = int( ceil( Ntracks / 10 ) )
    reorder_playlist_by_chunks( playlist_ID, 5, Nmoves, verbose = verbose )
    sleep( phasePause_s )
    ## Phase 3: 1 at a Time, 1/3 ##
    if verbose:
        print("\nPhase 3: 1 at a Time, 1/3\n")
    Nmoves = int( ceil( Ntracks / 3 ) )
    reorder_playlist_by_chunks( playlist_ID, 1, Nmoves, verbose = verbose )

# Shuffle Playlists!

In [3]:
playlist = {
    'study01' : "0a2qoe6S7lYeZ6nlhZdA0v",
    'study02' : "6gbtR2cBq5PvkghidCvvGk",
    'study03' : "3o3lN2qntdEV7UKTuuC77K",
    'study04' : "41sFSisljvBDMBXtpp5NIw",
    'study05' : "02iS5AFGp8YVuUUqcQf8ys",
    'study06' : "6KI7A4MWrSM7EyKRUjxIi1",
    'study07' : "3V055Md2JdrUT8tX0af7di",
    'study08' : "0tspdJlwSgiyf2O9PO6QaP",
    'study09' : "5mHRBFoQtYy2izeZ66pG95",
    'study10' : "3832xeKGEOAXFJqE4K8kIq",
    'study11' : "65MXR4dubPL9t0P4dgTWvn",
    'study12' : "0ecSAfnD4CulIVnLt26ukI",
    'study13' : "7K9ucByFRgDuZk8KMHeJkL",
    'study14' : "0v26bHydUxcGC5EbMlkjzG",
    'study15' : "6SqlfurCBP7eeMOojaDNtS",
    'study16' : "5TtKaKCouyJp7Hhtu4YlYm",
    'study17' : "5qX1Tq3IQ74iSgrlMfRhty",
    'study18' : "1bwXMYoRgEFWebWZ9ZPBqs",
    'study19' : "3ackEOD2vox5Oc1vjeALKJ", 
    'study20' : "3Cmpe4nGzOQDefowW49pBS",
    'PianoJz' : "2IqcxvGty13ewL583z9Kdn",
    'cringe1' : "2AAUYlKM1nXKHkZUpUSFbv",
}

complete_reorder_entire_playlist( playlist['study10'], verbose = True, phasePause_s = 5 )


Phase 1: Chunks of 100

Iteration 1, 100-200 --> 378:
	{'snapshot_id': 'AAAEmzPYfxY50edT2/nFC3vq1MMIcufA'}
Iteration 2, 300-400 --> 167:
	{'snapshot_id': 'AAAEnB+ne7YaCG9eiGtvYXUAs2/0mZ3a'}
Iteration 3, 300-400 --> 39:
	{'snapshot_id': 'AAAEnXsLGUwGEBQhSkzz5kEotBWb/QtI'}
Iteration 4, 100-200 --> 383:
	{'snapshot_id': 'AAAEnqHZIZBYOD9mqcR4SqIy5CBeRoV4'}

Phase 2: Chunks of 10

Iteration 1, 220-230 --> 346:
	{'snapshot_id': 'AAAEn2E3EpJaPgDYgSEXcgzrE7NDZbZa'}
Iteration 2, 230-240 --> 156:
	{'snapshot_id': 'AAAEoGU2eM7/MzTIW06VkKOMDc2xUfN8'}
Iteration 3, 40-50 --> 266:
	{'snapshot_id': 'AAAEoT0Wo3vz/z4DiID+kOj6GrEsnk/T'}
Iteration 4, 180-190 --> 95:
	{'snapshot_id': 'AAAEopbQqFN8StD8C/8Xnh07dYLIxVgh'}
Iteration 5, 290-300 --> 133:
	{'snapshot_id': 'AAAEow9N7WvFGsx0xHFb1BWDFd9HoPtL'}
Iteration 6, 270-280 --> 82:
	{'snapshot_id': 'AAAEpBAI8I/E9ed/xcVXRpGShJd2q8p3'}
Iteration 7, 330-340 --> 336:
	{'snapshot_id': 'AAAEpRUqb2oBSszr7BjZBndkeHeqKr8F'}
Iteration 8, 280-290 --> 274:
	{'snapshot_i

HTTP Error for PUT to https://api.spotify.com/v1/playlists/3832xeKGEOAXFJqE4K8kIq/tracks with Params: {} returned 400 due to Tracks selected to be reordered are out of bounds


Iteration 32, 90-100 --> 261:
	{'snapshot_id': 'AAAEvo17fsfaZHghKk8ypoIjz5m2YYDV'}
ERROR, 400-405 --> 30:
	http status: 400, code:-1 - https://api.spotify.com/v1/playlists/3832xeKGEOAXFJqE4K8kIq/tracks:
 Tracks selected to be reordered are out of bounds, reason: None
Iteration 34, 240-250 --> 109:
	{'snapshot_id': 'AAAEv4wU6aQMKPpl+T9d3Pn2gbbtd59w'}


HTTP Error for PUT to https://api.spotify.com/v1/playlists/3832xeKGEOAXFJqE4K8kIq/tracks with Params: {} returned 400 due to Tracks selected to be reordered are out of bounds


ERROR, 400-405 --> 81:
	http status: 400, code:-1 - https://api.spotify.com/v1/playlists/3832xeKGEOAXFJqE4K8kIq/tracks:
 Tracks selected to be reordered are out of bounds, reason: None
Iteration 36, 220-230 --> 79:
	{'snapshot_id': 'AAAEwEpf85MbBEPWRonccGML91kpv9r5'}
Iteration 37, 170-180 --> 215:
	{'snapshot_id': 'AAAEwRsfqAN6xBeqSvS09AfQgm01cU35'}
Iteration 38, 130-140 --> 341:
	{'snapshot_id': 'AAAEwgd4ni0wmbyXUiVrNHwFgRrQ8W3J'}
Iteration 39, 380-390 --> 82:
	{'snapshot_id': 'AAAEw4wR3oCs1DMmP26DHgIk0OyQ2UCc'}
Iteration 40, 230-240 --> 89:
	{'snapshot_id': 'AAAExLy4KkvuoiHv0S8onF/2iMwM8agp'}

Phase 3: Chunks of 5, 1/2

Iteration 1, 330-335 --> 377:
	{'snapshot_id': 'AAAExTtd67LEN3uJsl+xL3J+2QAuJtlE'}
Iteration 2, 30-35 --> 346:
	{'snapshot_id': 'AAAExnplk/J+8RlPPPgNGpdJrnZIfsJ9'}
Iteration 3, 230-235 --> 362:
	{'snapshot_id': 'AAAEx3721MQIBrC+REX+ljXqrzvmxhXN'}
Iteration 4, 360-365 --> 41:
	{'snapshot_id': 'AAAEyJLiQG1sA3j8K6k/SEIsyYLZJ608'}
Iteration 5, 120-125 --> 280:
	{'snapshot